**Alter working directory to get classes from parent folder**

In [1]:
import sys
import os

In [2]:
# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory (Job_Category_Classification)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

sys.path.append(parent_dir)

**Establish Database Connection**

In [3]:
import yaml
from scripts.SqlConn import SqlConn
from train_model.TrainModel import TrainModel

In [16]:
def load_config(file_path):
    full_path = os.path.join(parent_dir, file_path)
    with open(full_path,'r') as f:
        config = yaml.safe_load(f)
    return config

In [17]:
config = load_config('./configuration/config.yaml')
db_host = config['local_db']['host']
db_port = config['local_db']['port']
database = config['local_db']['database']
username = config['local_db']['username']
password = config['local_db']['password']

In [18]:
# Create database connection instance
sqlconn_obj = SqlConn(username,password,db_host,db_port,database)
engine = sqlconn_obj.connect()

**Train Model**

**Save Model**